In [123]:
'''
to do:
[]reduce the number of features in the set, remove features that are probably highly correlated
[]use the whole dataset since we're not predicting - try splitting with kfold of 5 or 10, use cross validation to check the model
[]multiply death feature by 100?
'''

"\nto do:\n[]reduce the number of features in the set, remove features that are probably highly correlated\n[]use the whole dataset since we're not predicting - try splitting with kfold of 5 or 10, use cross validation to check the model\n[]multiply death feature by 100?\n"

In [124]:
#import stuff
import numpy as np
import sklearn as sk
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [125]:
#import cleaned up datasets
protein_df = pd.read_csv("data/data_clean/protein_data_clean.csv")
kg_df = pd.read_csv("data/data_clean/kg_data_clean.csv")
kcal_df = pd.read_csv("data/data_clean/kcal_data_clean.csv")
fat_df = pd.read_csv("data/data_clean/fat_data_clean.csv")

In [126]:
#To aid regression, drop features that probably highly correlate with other features
protein_df = protein_df.drop(['Animal fats_p','Aquatic Products, Other_p', 'Sugar Crops_p', 'Vegetal Products_p', 'Vegetable Oils_p', 'Miscellaneous_p'],axis=1)
kg_df = kg_df.drop(['Animal fats_kg','Aquatic Products, Other_kg', 'Sugar Crops_kg', 'Vegetal Products_kg', 'Vegetable Oils_kg', 'Miscellaneous_kg'],axis=1)
kcal_df = kcal_df.drop(['Animal fats_kcal','Aquatic Products, Other_kcal', 'Sugar Crops_kcal', 'Vegetal Products_kcal', 'Vegetable Oils_kcal', 'Miscellaneous_kcal'],axis=1)
fat_df = fat_df.drop(['Animal fats_f','Aquatic Products, Other_f', 'Sugar Crops_f', 'Vegetal Products_f', 'Vegetable Oils_f', 'Miscellaneous_f'],axis=1)

In [127]:
#sorted(sk.metrics.SCORERS.keys())
linear_model.Lasso().get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'])

<h3>
I want to figure out what the lambda best value to penalize our model is for LASSO, for each dataset set.
I will also print out the weight vector, so we can determine what features LASSO determines are actually important.
</h3>

In [128]:
#gonna try different scoring metrics
def loop_lasso_mse(X_train, X_test, y_train, y_test, x):
    best_a = 0.1
    best_score = 0
    test_a_values = np.arange(0.1,11,0.1)
    for a in range(0,len(test_a_values)):
        Lreg = linear_model.Lasso(alpha = test_a_values[a])
        Lreg.fit(X_train, y_train)
        score = cross_val_score(Lreg, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
        if(np.average(score)<best_score):
            best_score=np.average(score)
            best_a = test_a_values[a]
    print("Best score :",best_score)
    print("Best Alpha :",best_a)
    #Get the best model's coefs
    Lreg = linear_model.Lasso(alpha = best_a)
    Lreg.fit(X_train, y_train)
    print("Weight coefficients:")
    for c in range(1,len(Lreg.coef_)):
        if(Lreg.coef_[c]>0.0):
            print(x.columns[c],":",Lreg.coef_[c])
    print('\n')

In [129]:
def loop_lasso_rmse(X_train, X_test, y_train, y_test, x):
    best_a = 0.1
    best_score = 0
    test_a_values = np.arange(0.1,11,0.1)
    for a in range(0,len(test_a_values)):
        Lreg = linear_model.Lasso(alpha = test_a_values[a])
        Lreg.fit(X_train, y_train)
        score = cross_val_score(Lreg, X_test, y_test, cv=5, scoring='neg_root_mean_squared_error')
        if(np.average(score)<best_score):
            best_score=np.average(score)
            best_a = test_a_values[a]
    print("Best score :",best_score)
    print("Best Alpha :",best_a)
    #Get the best model's coefs
    Lreg = linear_model.Lasso(alpha = best_a)
    Lreg.fit(X_train, y_train)
    print("Weight coefficients:")
    for c in range(1,len(Lreg.coef_)):
        if(Lreg.coef_[c]>0.0):
            print(x.columns[c],":",Lreg.coef_[c])
    print('\n')

In [132]:
def LASSO(df):
    df_x = df.drop(['Country', 'Population','Active','Recovered','Deaths','Confirmed','Undernourished'],axis=1)
    #to help regression, drop some features that are probably highly correlated
    #scale the data
    x = MinMaxScaler().fit_transform(df_x)
    print("Confirmed:")
    #26 is the start of our y features in the original dataset
    #20 is where they start with the extra features dropped
    #confirmed
    y = df.iloc[:,20].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    print("RMSE")
    loop_lasso_rmse(X_train, X_test, y_train, y_test, df_x)
    print("MSE")
    loop_lasso_mse(X_train, X_test, y_train, y_test, df_x)
    print("Deaths:")
    #deaths
    y = df.iloc[:,21].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    print("RMSE")
    loop_lasso_rmse(X_train, X_test, y_train, y_test, df_x)
    print("MSE")
    loop_lasso_mse(X_train, X_test, y_train, y_test, df_x)
    print("Recovered:")
    #recovered
    y = df.iloc[:,22].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    print("RMSE")
    loop_lasso_rmse(X_train, X_test, y_train, y_test, df_x)
    print("MSE")
    loop_lasso_mse(X_train, X_test, y_train, y_test, df_x)

In [133]:
LASSO(protein_df)

Confirmed:
RMSE
Best score : -2.265650245121237
Best Alpha : 0.5
Weight coefficients:


MSE
Best score : -5.688202101562668
Best Alpha : 0.5
Weight coefficients:


Deaths:
RMSE
Best score : -0.04280109245617871
Best Alpha : 0.1
Weight coefficients:


MSE
Best score : -0.0019074483033084128
Best Alpha : 0.1
Weight coefficients:


Recovered:
RMSE
Best score : -1.6726036625335319
Best Alpha : 0.30000000000000004
Weight coefficients:


MSE
Best score : -3.0772871659839103
Best Alpha : 0.30000000000000004
Weight coefficients:




In [ ]:
LASSO(fat_df)

In [ ]:
LASSO(kg_df)